# Lab 1. Forecasting Agent

## Introduction

In this notebook we show you how to create your first sub-agent on [Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/).

Amazon Bedrock Agents enable generative AI applications to execute multi-step business tasks using natural language.

In our first example we will create a forecasting agent, where customers can ask the agent to return information about their current energy consumption and forecast of it. 

The following represents the piece of architecture that will be built on this module.

![Forecast Agent Architecture](img/forecast_agent.png)

In this example, we will also enable our agent to use code intepretation capabilities to do basic calculations based on the energy usage and its forecast data. We are also using [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) to provide documentation about the forecasting model and its capabilities.

For completion reasons, we assume that the energy forecast has already been done outside the scope of this agent using a ML model hosted on SageMaker

## Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

In [1]:
!pip uninstall boto3 botocore awscli --yes

Found existing installation: boto3 1.40.39
Uninstalling boto3-1.40.39:
  Successfully uninstalled boto3-1.40.39
Found existing installation: botocore 1.40.39
Uninstalling botocore-1.40.39:
  Successfully uninstalled botocore-1.40.39
Found existing installation: awscli 1.42.39
Uninstalling awscli-1.42.39:
  Successfully uninstalled awscli-1.42.39


In [1]:
# Install latest dependencies
!python3 -m pip install --force-reinstall --no-cache -q -r ../requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.4.7 requires httpx<1,>=0.25.0, which is not installed.
sagemaker 2.251.1 requires numpy==1.26.4, but you have numpy 2.2.6 which is incompatible.
sagemaker 2.251.1 requires packaging<25,>=23.0, but you have packaging 25.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.3 which is incompatible.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


#### Restart kernel

If you face issues to apply the latest multi-agent capabilities, uncomment this line to restart kernel to ensure packages updates to take effect

In [2]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

Check your boto3 version

In [3]:
!pip freeze | grep boto3

boto3==1.40.55


#### Create Workshop ID

Next, you are going to create an ID for your resources that will be reused for the remainder of the workshop.

In [4]:
import uuid
import os
from pathlib import Path

def get_or_create_workshop_id():
    workshop_id_file = '../.workshop_id'
    
    if os.path.exists(workshop_id_file):
        with open(workshop_id_file, 'r') as f:
            return f.read().strip()
    else:
        workshop_id = str(uuid.uuid4())[:8]
        with open(workshop_id_file, 'w') as f:
            f.write(workshop_id)
        return workshop_id

workshop_id = get_or_create_workshop_id()
resource_suffix = f"{workshop_id}"
print("Your resource suffix is", resource_suffix)

Your resource suffix is 7eb12cba


## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and you will start to create your first agent.

In [33]:
import boto3
import os
import json
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
]

curr_month = datetime.now()

In [34]:
forecast_agent_name = f"forecast-{resource_suffix}"

forecast_lambda_name = f"fn-forecast-agent-{resource_suffix}"

forecast_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{forecast_agent_name}'

dynamodb_table = f"{forecast_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "day"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{forecast_agent_name}-kb'

knowledge_base_description = "KB containing information on how forecasting process is done"
bucket_name = f'forecast-agent-kb-{account_id}-{resource_suffix}'


### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [9]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and syncronize Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

This data contains basic information about how forecast process is done.

**This creation process can take several minutes.**

In [10]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Creating KB forecast-7eb12cba-kb
Step 1 - Creating or retrieving forecast-agent-kb-833005555478-7eb12cba S3 bucket for Knowledge Base documents
Creating bucket forecast-agent-kb-833005555478-7eb12cba
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_840) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '316',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Sat, 18 Oct 2025 12:54:57 '
                                                 'GMT',
                                         'x-amzn-requestid': 'bb357d92-97ef-45a2-bbf2-6bff9e115c5f'},
                        'HTTPStatusCode': 200,
                        'RequestId'

## Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [11]:
path = "kb_documents"

# Check whether the specified path exists or not
is_exist = os.path.exists(path)
if not is_exist:
   # Create a new directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

The kb_documents directory already exists!


Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [40]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            {
                "text": prompt
            }
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[1],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2048,
            "temperature": 0
        },
    )

    return response['output']['message']


def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close()

### Generating data prompt
Generating one file with forecasting info using the LLM model

In [35]:
text_generation_energy_instructions = '''
    You will be act as data-scientist that knows how to do machine learning
    forecasting using Python and scikit learn. You will generate a step-by-step
    on how to create a forecast process for a time-series data.

    This data has the following json structure:
    {
        "customer_id": "1",
        "day": "2024/06/01",
        "sumPowerReading": "120.0",
        "kind":"measured"
    }

    Choose one forecast algorithm, that works on scikit-learn, explain the
    details on how to create a step-by-step forecast, with code sample,
    showcasing how to run forecast on this data.

    Include some explanation on how to understand the forecasted values and
    how to decide the factors driving those values.

    Answer only with the step-by-step, avoid answer with afirmations like:
    "OK, I can generate it," or "Yes, please find following example."
    Be direct and only reply the step-by-step.
'''

solar_energy_file_name = 'forecasting-info.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)

print("Generated data to be stored in the KB:\n", response_message['content'][0]['text'])
write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    response_message['content'][0]['text']
)

Generated data to be stored in the KB:
 Step-by-step forecast process for time-series data using scikit-learn:

1. Import necessary libraries:
```python
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
```

2. Load and preprocess the data:
```python
# Load data from JSON file
df = pd.read_json('data.json', lines=True)

# Convert 'day' to datetime
df['day'] = pd.to_datetime(df['day'])

# Set 'day' as index
df.set_index('day', inplace=True)

# Sort by date
df.sort_index(inplace=True)

# Create lag features
for i in range(1, 8):
    df[f'lag_{i}'] = df['sumPowerReading'].shift(i)

# Drop rows with NaN values
df.dropna(inplace=True)
```

3. Prepare features and target:
```python
X = df[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7']]
y = df['sumPowerReadin

### Uploading data to s3
Uploading generated files into an Amazon S3 Bucket.

In [14]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [15]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

uploading file kb_documents/forecasting-info.txt to forecast-agent-kb-833005555478-7eb12cba
{ 'dataSourceId': 'IJJ9U4BSOZ',
  'ingestionJobId': 'KXOQPB4KHN',
  'knowledgeBaseId': 'EUDSVOIZ6T',
  'startedAt': datetime.datetime(2025, 10, 18, 13, 4, 10, 578462, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 10, 18, 13, 4, 10, 578462, tzinfo=tzlocal())}
{ 'dataSourceId': 'IJJ9U4BSOZ',
  'ingestionJobId': 'KXOQPB4KHN',
  'knowledgeBaseId': 'EUDSVOIZ6T',
  'startedAt': datetime.datetime(2025, 10, 18, 13, 4, 10, 578462, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
       

## Creating Agent

Create the forecast agent that will have an `Amazon Bedrock Knowledge Base` with information on how forecast process is done as well as the `action groups` to handle the user requests and `code interpretation` capabilities for calculating basic user's requests such as increase in consumption.

In order to have accurate agents, it is important to set unambiguous instructions of what the agent should do and what it should not do. It is also important to provide clear definitions for when the agent should use the knowledge bases and action groups available to it.

We will provide the following instructions to our agent:
```
You are an Energy Assistant that helps customers understand their energy consumption patterns and future usage expectations.

Your capabilities include:
1. Analyzing historical energy consumption
2. Providing consumption forecasts
3. Generating usage statistics
4. Updating forecasts for specific customers

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. Use code generation and interpretation capabilities for any on the fly calculation. DO NOT try to calculate things by yourself.
6. DO NOT plot graphs. Refuse to do so when asked by the user. Instead provide an overview of the data

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user
```

We will also connect a knowledge base for the explanation of the forecasting methodologies with the following instructions:
```
Access this knowledge base when needing to explain specific forecast generation methodology.
```

And we will make the following tool available to the agent:
- `get_forecasted_consumption`: Gets the next 3 months energy usage forecast
- `get_historical_consumption`: Gets energy usage history to date
- `get_consumption_statistics`: Gets current month usage analytics
- `update_forecasting`: Updates the energy forecast for a specific month


In [16]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [17]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Access this knowledge base when needing to explain specific forecast generation methodology."""
}

In [18]:
agent_description = """You are a energy usage forecast bot.
You can retrieve historical energy consumption, forecasted consumption, usage statistics and update a forecast for a specific user"""

agent_instruction = """You are an Energy Assistant that helps customers understand their energy consumption patterns and future usage expectations.

Your capabilities include:
1. Analyzing historical energy consumption
2. Providing consumption forecasts
3. Generating usage statistics
4. Updating forecasts for specific customers

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. Use code generation and interpretation capabilities for any on the fly calculation. DO NOT try to calculate things by yourself.
6. DO NOT plot graphs. Refuse to do so when asked by the user. Instead provide an overview of the data

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user"""

forecast_agent = agents.create_agent(
    forecast_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=True
)

forecast_agent

Waiting for agent status to change. Current status CREATING
Agent id EGYPPNBQRV current status: NOT_PREPARED


('EGYPPNBQRV',
 'TSTALIASID',
 'arn:aws:bedrock:us-east-1:833005555478:agent-alias/EGYPPNBQRV/TSTALIASID')

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [ ]:
agents.associate_kb_with_agent(
    forecast_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

### Creating Lambda

In order to enable the agent to execute tasks, we will create an AWS Lambda function that implements the tasks execution. We will then provide this lambda function to the agent action group. You can find more information on how to use action groups to define actions that your agent can perform [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html)

On this block, we're going to generate Lambda function Code:

In [21]:
%%writefile forecast.py
import boto3
import json
import os

from boto3.dynamodb.conditions import Key, Attr
from datetime import datetime
from decimal import Decimal

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')
truncated_month = datetime.today().replace(day=1, hour=0, minute=0, second=0, microsecond=0)


def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def trunc_datetime(month,year):
    return datetime.today().replace(year =int(year), month=int(month), day=1, hour=0, minute=0, second=0, microsecond=0)

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(
    table_name: str, 
    pk_field: str,
    pk_value: str,
    sk_field: str=None, 
    sk_value: str=None,
    attr_key: str=None,
    attr_val: str=None
):
    try:

        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_field:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).eq(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        if attr_key:
            attr_expression = Attr(attr_key).eq(attr_val)
            query_data = table.query(
                KeyConditionExpression=key_expression,
                FilterExpression=attr_expression
            )
        else:
            query_data = table.query(
                KeyConditionExpression=key_expression
            )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def get_forecasted_consumption(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="kind", attr_val="forecasted")

def get_historical_consumption(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="kind", attr_val="measured")

def get_consumption_statistics(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         dynamodb_sk, 
                         truncated_month.strftime('%Y/%m/%d'))

def update_forecasting(customer_id, month, year, usage):
    current_date = trunc_datetime(month, year)
    if  current_date >= truncated_month:
        item = {
            'customer_id': customer_id,
            'day': current_date.strftime('%Y/%m/%d'),
            'sumPowerReading': Decimal(usage),
            'kind': 'forecasted'
        }
        put_dynamodb(dynamodb_table, item)
        return "Day: {} updated for customer: {}".format(current_date.strftime('%Y/%m/%d'), customer_id)
    else:
        return "You're trying to change a past date: {} for customer: {}, which is not allowed".format(current_date.strftime('%Y/%m/%d'), customer_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'get_forecasted_consumption':
        result = get_forecasted_consumption(customer_id)
    elif function == 'get_historical_consumption':
        result = get_historical_consumption(customer_id)
    elif function == 'get_consumption_statistics':
        result = get_consumption_statistics(customer_id)
    elif function == 'update_forecasting':
        month = get_named_parameter(event, "month")
        year = get_named_parameter(event, "year")
        usage = get_named_parameter(event, "usage")
        result = update_forecasting(customer_id, month, year, usage)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Overwriting forecast.py


### Defining available actions

Next we will define the available actions that an agent can perform using [Function Details](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html). You can also do this task using OpenAPI Schemas, which can be very useful if you already have an OpenAPI schema available for your application.

When creating your function details, it is important to provide clear descriptions for the function and for its parameters, as your agent depends on them to correctly orchestrate the tasks to be executed

In [22]:
functions_def = [
    {
        "name": "get_forecasted_consumption",
        "description": """Gets the next 3 months energy usage forecast""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_historical_consumption",
        "description": """Gets energy usage history to date""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_consumption_statistics",
        "description": """Gets current month usage analytics""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "update_forecasting",
        "description": """Updates the energy forecast for a specific month""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "month": {
                "description": "Target update month. In the format MM",
                "required": True,
                "type": "integer"
            },
            "year": {
                "description": "Target update year. In the format YYYY",
                "required": True,
                "type": "integer"
            },
            "usage": {
                "description": "New consumption value",
                "required": True,
                "type": "integer"
            }
        }
    }
]

### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [23]:
agents.add_action_group_with_lambda(
    agent_name=forecast_agent_name,
    lambda_function_name=forecast_lambda_name,
    source_code_file="forecast.py",
    agent_functions=functions_def,
    agent_action_group_name="forecast_consumption_actions",
    agent_action_group_description="Function to get usage forecast for a user ",
    dynamo_args=dynamoDB_args
)

## Loading data to DynamoDB

Now that we've created our agent, let's load some generated data to DynamoDB. That will allow the agent to interact with some live data to perform actions

In [24]:
agents.generate_fake_data_dynamodb()

In [25]:
with open("1_user_sample_data.json") as f:
    table_items = [json.loads(line) for line in f]
    
agents.load_dynamodb(dynamodb_table, table_items)

Testing that data was loaded on DynamoDB

In [26]:
today_query = f"{curr_month.year}/{curr_month.month:02d}/01"

resp = agents.query_dynamodb(
    dynamodb_table, dynamodb_pk, '1', dynamodb_sk, today_query
)
resp

[{'customer_id': '1',
  'kind': 'measured',
  'sumPowerReading': '156',
  'day': '2025/10/01'}]

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing get forecast
First let's test the get forecast action

In [29]:
%%time
response = agents.invoke(
    """can you give me my forecasted energy consumption? 
    How does it compare with my past energy usage? My customer id is 1""", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 9c974c70-353b-4215-abc1-f8ab3b3ff48b
invokeAgent API session ID: 7d55492e-ac21-11f0-8d13-16ffda0c90af
---- Step 1 ----
Took 5.2s, using 2156 tokens (in: 1889, out: 267) to complete prior action, observe, orchestrate.
To answer your question comprehensively, I'll need to retrieve three sets of data:
1. Your forecasted energy consumption for the next 3 months
2. Your historical energy consumption
3. Your current month's consumption statistics for comparison

I'll use the functions available to gather this information simultaneously.
Using tool: get_forecasted_consumption with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'kind': 'forecasted', 'sumPowerReading': '124', 'day': '2025/11/01'}, {'customer_id': '1', 'kind': 'forecasted', 'sumPowerReading': '142', 'day': '2025/12/01'}, {'customer_id': '1', 'kind': 'forecasted', 'sumPowerReading': '164', 'day': '2026/01/01'}, {'customer_id': '1', 'kind..

Exception: ('Unexpected exception: ', EventStreamError('An error occurred (throttlingException) when calling the InvokeAgent operation: Your request rate is too high. Reduce the frequency of requests. Check your Bedrock model invocation quotas to find the acceptable frequency.'))

In [36]:
time.sleep(60)

### Testing get historical consumption
Now we can test the historical energy consumption and also use code interpretation calculate the average energy spending in summer months

In [37]:
%%time
response = agents.invoke(
    "can you give me my past energy consumption? What is my average spending on summer months? My customer id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: f26ded17-c8d5-4ace-8677-40d173326522
invokeAgent API session ID: 7d55492e-ac21-11f0-8d13-16ffda0c90af
---- Step 1 ----
Took 5.4s, using 2120 tokens (in: 1884, out: 236) to complete prior action, observe, orchestrate.
To answer your questions, I'll need to retrieve your historical consumption data and then analyze it to calculate your average spending during summer months. I'll use the following functions:
1. get_historical_consumption to fetch your past energy consumption
2. get_consumption_statistics to get current month's usage analytics, which might provide additional insights

I'll call both functions simultaneously to optimize the process.
Using tool: get_historical_consumption with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'kind': 'measured', 'sumPowerReading': '122', 'day': '2025/06/01'}, {'customer_id': '1', 'kind': 'measured', 'sumPowerReading': '192', 'day': '2025/07/01'}, {'custo

Exception: ('Unexpected exception: ', EventStreamError('An error occurred (throttlingException) when calling the InvokeAgent operation: Your request rate is too high. Reduce the frequency of requests. Check your Bedrock model invocation quotas to find the acceptable frequency.'))

In [38]:
time.sleep(60)

In [39]:
forecast_agent[0]

'EGYPPNBQRV'

### Testing knowledge base access
Now let's check the knowledge base access by asking a question about the forecasting algorithm

In [ ]:
%%time
response = agents.invoke(
    "What's algorithm used for forecast?", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

### Testing forecast update
Now we can test the functionality to update the expected forecasting

In [ ]:
%%time

future_2m = curr_month + relativedelta(months=2)
future_2m_formatted = future_2m.strftime("%Y/%m")

response = agents.invoke(
    f"Can you update my forecast for month {future_2m_formatted}? I will be travelling and my estimate will be 50. My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

### Confirming that forecast was updated
After updating our forecast, let's check that the forecast was updated and plot a new graph

In [ ]:
%%time
response = agents.invoke(
    "Can you give me my forecasted energy consumption month by month? My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

### Testing forecasting statistics
Finally, let's test the get stats functionality

In [ ]:
%%time
response = agents.invoke(
    "can you give me my current consumption? My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [41]:
forecast_agent_alias_id, forecast_agent_alias_arn = agents.create_agent_alias(
    forecast_agent[0], 'v1'
)

## Saving information
Let's store some environment variables to be used on our next notebooks.

In [42]:
forecast_agent_arn = agents.get_agent_arn_by_name(forecast_agent_name)
forecast_agent_id = forecast_agent[0]
forecast_kb = knowledge_base_name
forecast_dynamodb = dynamodb_table

%store forecast_agent_arn
%store forecast_agent_alias_arn
%store forecast_agent_alias_id
%store forecast_lambda_name
%store forecast_agent_name
%store forecast_agent_id
%store forecast_kb
%store forecast_dynamodb

Stored 'forecast_agent_arn' (str)
Stored 'forecast_agent_alias_arn' (str)
Stored 'forecast_agent_alias_id' (str)
Stored 'forecast_lambda_name' (str)
Stored 'forecast_agent_name' (str)
Stored 'forecast_agent_id' (str)
Stored 'forecast_kb' (str)
Stored 'forecast_dynamodb' (str)


In [43]:
forecast_agent_arn, forecast_agent_alias_arn, forecast_agent_alias_id

('arn:aws:bedrock:us-east-1:833005555478:agent/EGYPPNBQRV',
 'arn:aws:bedrock:us-east-1:833005555478:agent-alias/EGYPPNBQRV/I1KJYFXMQJ',
 'I1KJYFXMQJ')

## Next Steps
Congratulations! We've now created a forecasting agent. Next we will create our solar panel agent